# Import packages

In [ ]:
import numpy as np
import torch
import math
import torch.nn as nn
import torch.nn.functional as f
import random
from tqdm import tqdm

# Main objective function

In [ ]:
class main_obj_function(object):
    def __init__(self, theta=[0.05884, 4.298, 21.8]):
        self.theta = theta
    
    def sub_obj_function(self, x, w):
        value1 = x * self.theta[2] * math.exp(-self.theta[0] * x)
        value2 = x * self.theta[2] * math.exp(-self.theta[1] * x)
        value3 = math.exp(-self.theta[1] * x) - math.exp(-self.theta[0] * x)
        value = torch.Tensor([[value1],[value2],[value3]])
        return w * (value @ value.T)
    
    def forward(self, xs_and_ws):
        xs = xs_and_ws[:3]
        ws = xs_and_ws[3:]
        result = torch.stack(list(map(self.sub_obj_function,xs,ws))).sum(axis=0).det()
        return result.item()

# Differential Evolution algorithm

In [ ]:
class Diff_Evo(object):
    def __init__(self, pop_size=100, weight_F=1, cross_proby=0.5):
        self.pop_size = pop_size
        self.weight_F = weight_F
        self.cross_proby = cross_proby
        self.softmax = nn.Softmax(dim=0)
        self.main_obj_function = main_obj_function()
        
    def forward(self, max_iteration=200):
        global_best_each_iter = []
        point_list = list(map(self.random_generate_sampe,list(range(self.pop_size))))
        for t in tqdm(range(max_iteration)):
            if t % 2 == 0:
                function_value = list(map(self.main_obj_function.forward,point_list))
#                 print('itr : '+str(t)+', value = '+str(max(function_value)))

            new_point_list = list(map(self.get_new_point,
                                      list(range(self.pop_size)),
                                      [point_list for i in range(self.pop_size)]))
            point_list = list(map(self.update_point, point_list, new_point_list))
        
            global_best_each_iter.append(max(list(map(self.main_obj_function.forward,point_list))))
        # return final_value
        final_value = list(map(self.main_obj_function.forward,point_list))
        final_value_index = final_value.index(max(final_value))
        # global_best_each_iter.append(final_value)
        return dict({'max_value':max(final_value),
                     'parameter':point_list[final_value_index],
                     'global_best_each_iter':global_best_each_iter})
    
    def random_generate_sampe(self, *arg):
        # torch.manual_seed(2609)
        random_x = (torch.rand(3) * 30).sort()[0]
        random_w = f.normalize(torch.rand(3), p=1, dim=0)
        init_point = torch.cat((random_x, random_w))
        return init_point
    
    def get_new_point(self, index, points, update_type='exp'):
        point_1, point_2, point_3 = random.sample(points[:index] + points[index+1:], 3)
        velocity = point_1 + self.weight_F * (point_2 - point_3)
        new_point = points[index].clone()
        min_value, max_value = sorted(random.sample(range(6),2))
        max_value += 1
        new_point[min_value:max_value] = velocity[min_value:max_value]
        for i in range(3):
            if new_point[i] < 0:
                new_point[i] = 0
            elif new_point[i] > 30:
                new_point[i] = 30
        new_point[0:3] = new_point[0:3].sort()[0]
        new_point[3:6] = self.softmax(new_point[3:6])
        return new_point
    
    def update_point(self, point_1, point_2):
        function_value_1 = self.main_obj_function.forward(xs_and_ws=point_1)
        function_value_2 = self.main_obj_function.forward(xs_and_ws=point_2)
        final_point = point_1 if function_value_1 > function_value_2 else point_2
        return final_point

# Particle Swarm Optimization

In [ ]:
class PSO(object):
    def __init__(self, pop_size=50, alpha=2, beta=2, velocity_theta=0.7):
        self.pop_size = pop_size
        self.alpha = alpha
        self.beta = beta
        self.velocity_theta = velocity_theta
        self.softmax = nn.Softmax(dim=0)
        self.main_obj_function = main_obj_function()
    
    def forward(self, max_iteration=200):
        global_best_each_iter = []
        
        point_list = list(map(self.random_generate_sampe,list(range(self.pop_size))))
        x_list = [sub_data[0] for sub_data in point_list]
        v_list = [sub_data[1] for sub_data in point_list]
        
        local_best_point = x_list[:]
        function_value = list(map(self.main_obj_function.forward,x_list))
        global_best_value = max(function_value)
        function_value_index = function_value.index(global_best_value)
        global_best_point = x_list[function_value_index]
        
        for t in tqdm(range(max_iteration)):
#             if t % 10 == 0:
#                 print('itr : '+str(t)+', global best value = '+str(global_best_value))
                
            new_v_list = list(map(self.eval_velocity,v_list,x_list,local_best_point,[global_best_point for i in range(self.pop_size)]))
            x_list = list(map(self.get_new_point,new_v_list,x_list))
            
            local_best_point = list(map(self.update_local_best_point,local_best_point,x_list))

            function_value = list(map(self.main_obj_function.forward,local_best_point))
            function_value_index = function_value.index(max(function_value))
            sectional_global_best_point = local_best_point[function_value_index]
            sectional_global_best_value = max(function_value)
            global_best_point = sectional_global_best_point if sectional_global_best_value > global_best_value else global_best_point
            global_best_value = sectional_global_best_value if sectional_global_best_value > global_best_value else global_best_value
            v_list = new_v_list[:]
            global_best_each_iter.append(global_best_value)
        
        return dict({'max_value':global_best_value,
                     'parameter':global_best_point,
                     'global_best_each_iter':global_best_each_iter})
    
    def random_generate_sampe(self, *arg):
        # torch.manual_seed(4022)
        random_x = (torch.rand(3) * 30).sort()[0]
        random_w = f.normalize(torch.rand(3), p=1, dim=0)
        init_point = torch.cat((random_x, random_w))
        init_volocity = torch.Tensor([0, 0, 0, 0, 0, 0])
        return init_point, init_volocity
    
    def eval_velocity(self, v_t, x_t, local_bast, global_bast):
        local_v = self.beta * torch.rand(6) * (local_bast - x_t)
        global_v = self.alpha * torch.rand(6) * (global_bast - x_t)
        v_t_plus_1 = self.velocity_theta * v_t + local_v + global_v
        return v_t_plus_1
    
    def get_new_point(self, v_t_plus_1, x_t):
        x_t_plus_1 = x_t + v_t_plus_1
        for i in range(3):
            if x_t_plus_1[i] < 0:
                x_t_plus_1[i] = 0
            elif x_t_plus_1[i] > 30:
                x_t_plus_1[i] = 30
        x_t_plus_1[0:3] = x_t_plus_1[0:3].sort()[0]
        x_t_plus_1[3:6] = self.softmax(x_t_plus_1[3:6])
        return x_t_plus_1
    
    def update_local_best_point(self, point_1, point_2):
        function_value_1 = self.main_obj_function.forward(xs_and_ws=point_1)
        function_value_2 = self.main_obj_function.forward(xs_and_ws=point_2)
        final_point = point_1 if function_value_1 > function_value_2 else point_2
        return final_point

# Firefly algorithm

In [ ]:
class FA(object):
    def __init__(self, pop_size=30, alpha=1, beta=1, gamma=0.1, theta=0.95):
        self.pop_size = pop_size
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.theta = theta
        self.softmax = nn.Softmax(dim=0)
        self.main_obj_function = main_obj_function()
    
    def forward(self, max_iteration=100):
        all_point_list = []
        global_best_each_iter = []
        init_point_list = list(map(self.random_generate_sampe,list(range(self.pop_size))))
        all_point_list.append(init_point_list)
        
        function_values = list(map(self.main_obj_function.forward,init_point_list))
        global_best_value = max(function_values)
        
        range_index = list(range(len(init_point_list)))
        for t in tqdm(range(max_iteration)):
            
            if 'temp_point_list' not in locals():
                temp_point_list = init_point_list[:]
            
#             if t % 5 == 0 and t != 0:
#                 print(global_best_value)
                
            for i in range_index:
                inner_index = range_index[:]
                inner_index.remove(i)

                for j in inner_index:
                    if function_values[i] < function_values[j]:
                        
                        attractive = self.eval_attractive(self.eval_distence(temp_point_list[i], temp_point_list[j]))
                        velocity = attractive * (temp_point_list[j] - temp_point_list[i])
                        x_update = temp_point_list[i] + velocity + self.alpha * (self.theta ** t) * (2*torch.rand(6) - 1)
                        for k in range(3):
                            if x_update[k] < 0:
                                x_update[k] = 0
                            elif x_update[k] > 30:
                                x_update[k] = 30
                        x_update[0:3] = x_update[0:3].sort()[0]
                        x_update[3:6] = self.softmax(x_update[3:6])
                    else:
                        x_update = temp_point_list[i]
                        
                    function_value = self.main_obj_function.forward(x_update)
                    # function_values[i] = function_value
                    temp_point_list[i] = x_update
                    
            function_values = list(map(self.main_obj_function.forward,temp_point_list))
            global_best_value = max(function_values) # global_best_value if global_best_value > function_value else function_value
            global_best_each_iter.append(global_best_value)
            # all_point_list.append(temp_point_list)
                        
        global_input_index = function_values.index(global_best_value)
        # return dict({})global_best_value, temp_point_list[global_input_index], all_point_list
        return dict({'max_value':global_best_value,
                     'parameter':temp_point_list[global_input_index],
                     'global_best_each_iter':global_best_each_iter})
    
    def random_generate_sampe(self, *arg):
        # torch.manual_seed(4022)
        random_x = (torch.rand(3) * 30).sort()[0]
        random_w = f.normalize(torch.rand(3), p=1, dim=0)
        init_point = torch.cat((random_x, random_w))
        return init_point
    
    def eval_attractive(self, distence):
        return self.beta / (1 + self.gamma * (distence)**2)
        
    def eval_distence(self, point1, point2):
        diff = point1 - point2
        distence = torch.sqrt(diff @ diff.T)
        return distence